In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from scipy.stats import ttest_ind
from scipy.stats import ttest_ind_from_stats
from scipy.stats import t
import seaborn as sns
import scipy.stats as stats
from patsy import build_design_matrices

In [ ]:
df = pd.read_csv("../Data/fertility_filtered.csv", low_memory=False)
# df = pd.read_csv("../Data/fertilityDF_W_MY_filtered.csv", low_memory=False)
# df = pd.read_csv("../Data/CowData/fertilityDF_W_MY.csv", low_memory=False)
# df = df.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(df.shape)

In [ ]:
df

In [ ]:
# Check data distribution in filtered dataset
print(df.groupby(["HeatStress", "Parity", "Breed"]).size())

# CR: GLMM
Note: Doesn't include random effects cuz reasons

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import product
from scipy.stats import ttest_ind
import statsmodels.api as sm

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CR0", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# Define the formula for the generalized linear mixed model
formula = "CR0 ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the GLMM with HYS as a random effect
# Here we use GLM with a binomial family and a logit link function
model = smf.glm(
    formula, 
    data=df,
    family=sm.families.Binomial(link=sm.families.links.logit()),  # Binomial outcome with logit link
    offset=None
)

# Fit the model
result = model.fit()
print(result.summary())

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.params
print("Fixed effects:\n", fixed_effects)

# Extract standard errors for the fixed effects
se_fixed_effects = result.bse
print("\nStandard errors for fixed effects:\n", se_fixed_effects)

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination using the fitted model
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_CR0"] = predictions

# Calculate LS-means: Average predicted CR0 for each combination of HeatStress, Parity, and Breed
ls_means = prediction_df.groupby(["HeatStress", "Parity", "Breed"])["Predicted_CR0"].mean()

# Print the LS-means
print("LS-means estimates:\n", ls_means)

# Save LS-means to CSV
ls_means.to_csv("CR_LSM.csv")

# Calculate standard errors for LS-means for each fixed effect level (HeatStress, Parity, Breed)
# We will use the standard errors for each fixed effect extracted above.

# For HeatStress
se_heatstress = se_fixed_effects.get("C(HeatStress)[T." + str(heatstress_levels[1]) + "]")  # Adjust for first level comparison
print(f"Standard Error for HeatStress: {se_heatstress}")

# For Parity
se_parity = se_fixed_effects.get("C(Parity)[T." + str(parity_levels[1]) + "]")  # Adjust for first level comparison
print(f"Standard Error for Parity: {se_parity}")

# For Breed
se_breed = se_fixed_effects.get("C(Breed)[T." + str(breed_levels[1]) + "]")  # Adjust for first level comparison
print(f"Standard Error for Breed: {se_breed}")

# Pairwise comparisons (PDIFF equivalent in SAS)

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_CR0"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_CR0"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)

# --- Examining Residuals ---

# Calculate residuals (difference between observed and predicted values)
residuals = df["CR0"] - result.predict()

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# CR: LMM
However, residuals are sucky cuz binary trait

To analyze CR0 and get separate estimates of LS-means for HeatStress, Parity and Breed with SE

In [ ]:
# --- PREP ---

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CR0", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# --- FIT MODEL ---

# Define the formula for the mixed model
formula = "CR0 ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(
    formula, 
    df, 
    groups=df["HYS"],  # Top-level random effect
    re_formula="~1",   # Random intercept
    vc_formula={"SE_Number": "0 + C(SE_Number)"}  # Nested random effect
)

# Fit the model
result = model.fit()
print(result.summary())

# --- ESTIMATE LS-MEANS ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
print("Fixed effects:\n", fixed_effects)

# Extract the covariance matrix of fixed effects
cov_matrix = result.cov_re

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_CR0"] = predictions

# 1. LS-means for HeatStress
ls_means_heatstress = prediction_df.groupby("HeatStress")["Predicted_CR0"].mean()
print("\nLS-means estimates for HeatStress:\n", ls_means_heatstress)

# 2. LS-means for Parity
ls_means_parity = prediction_df.groupby("Parity")["Predicted_CR0"].mean()
print("\nLS-means estimates for Parity:\n", ls_means_parity)

# 3. LS-means for Breed
ls_means_breed = prediction_df.groupby("Breed")["Predicted_CR0"].mean()
print("\nLS-means estimates for Breed:\n", ls_means_breed)

# --- CALC SE ---

# Calculate standard errors for LS-means for each combination of HeatStress, Parity, and Breed
# This requires computing SE for each fixed effect level.

def calculate_se_for_factor(factor_name, factor_levels, fixed_effects, se_fixed_effects, reference_se=None):
    """
    Calculates standard errors for each level of a factor
    and returns them in a Series.
    """
    se_dict = {}
    for level in factor_levels:
        if level == factor_levels[0]:  # Assume the first level is the reference category
            # The SE for the reference category is the intercept SE
            se_dict[level] = reference_se
        else:
            # Construct the name of the fixed effect term for the factor
            effect_name = f"C({factor_name})[T.{level}]"
            # Extract the standard error for the fixed effect term
            se_value = se_fixed_effects.get(effect_name, np.nan)  # Default to NaN if not found
            se_dict[level] = se_value

    return pd.Series(se_dict, name=f"SE_{factor_name}")

# Calculate SE for each factor level
se_heatstress = calculate_se_for_factor("HeatStress", heatstress_levels, fixed_effects, se_fixed_effects, intercept_se)
se_parity = calculate_se_for_factor("Parity", parity_levels, fixed_effects, se_fixed_effects, intercept_se)
se_breed = calculate_se_for_factor("Breed", breed_levels, fixed_effects, se_fixed_effects, intercept_se)

# Print SE for each factor level
print("\nStandard Errors for HeatStress:\n", se_heatstress)
print("\nStandard Errors for Parity:\n", se_parity)
print("\nStandard Errors for Breed:\n", se_breed)

"""
# Save SEs to CSV
se_heatstress.to_csv("HeatStress_SE.csv")
se_parity.to_csv("Parity_SE.csv")
se_breed.to_csv("Breed_SE.csv")
"""

# --- PAIRWISE COMPARISONS (PDIFF EQUIVALENT IN SAS) ---

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_CR0"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_CR0"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

"""
# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)
"""

# --- EXAMINING RESIDUALS ---

# Calculate residuals
residuals = result.resid

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

Include SE estimation for the nested fixed effects, and full pairwise comparisons for nested effects

In [ ]:
# --- PREP ---

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CR0", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# --- MODELLING ---
# Define the formula for the mixed model
formula = "CR0 ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(
    formula, 
    df, 
    groups=df["HYS"],  # Top-level random effect
    re_formula="~1",   # Random intercept
    vc_formula={"SE_Number": "0 + C(SE_Number)"}  # Nested random effect
)

# Fit the model
result = model.fit()
print(result.summary())

# --- LS-MEANS & SE ESTIMATION ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
se_fixed_effects = result.bse

# Intercept SE for reference category
intercept_se = se_fixed_effects.get("Intercept", np.nan)

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Predict CR0 values for each combination
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")
prediction_df["Predicted_CR0"] = result.predict(exog=prediction_df)

# Calculate SE for each combination of HeatStress, Parity, and Breed
def calculate_combination_se(row, fixed_effects, se_fixed_effects, intercept_se):
    """Calculate standard error for a specific combination of HeatStress, Parity, and Breed."""
    se_squared = intercept_se**2  # Start with intercept variance
    
    for factor_name in ["HeatStress", "Parity", "Breed"]:
        level = row[factor_name]
        if level != df[factor_name].cat.categories[0]:  # Check if not reference category
            effect_name = f"C({factor_name})[T.{level}]"
            se_squared += se_fixed_effects.get(effect_name, 0)**2  # Add variance of the level
    
    return np.sqrt(se_squared)  # Return standard error as sqrt of variance

prediction_df["SE"] = prediction_df.apply(
    calculate_combination_se, 
    axis=1, 
    fixed_effects=fixed_effects, 
    se_fixed_effects=se_fixed_effects, 
    intercept_se=intercept_se
)

# Format and save the output as "CR_LSM.csv"
cr_lsm = prediction_df.pivot_table(
    index=["HeatStress", "Parity", "Breed"], 
    values=["Predicted_CR0", "SE"]
).reset_index()

"""
# Save the LS-means and SEs
cr_lsm.to_csv("CR_LSM.csv", index=False)
"""

# Print the final output structure
print("\nGenerated CR_LSM.csv:")
print(cr_lsm)

# --- PAIRWISE COMPARISONS OF NESTED EFFECTS ---

def pairwise_nested_comparisons(ls_means_df):
    """
    Perform pairwise comparisons for all combinations of HeatStress, Parity, and Breed.
    """
    comparison_results = []
    for i, row1 in ls_means_df.iterrows():
        for j, row2 in ls_means_df.iterrows():
            if i < j:  # Compare each pair only once
                # Compute the difference between predicted means
                mean_diff = row1["Predicted_CR0"] - row2["Predicted_CR0"]
                
                # Compute the pooled SE for the difference
                pooled_se = np.sqrt(row1["SE"]**2 + row2["SE"]**2)
                
                # Compute t-statistic
                t_stat = mean_diff / pooled_se
                
                # Approximate degrees of freedom (Satterthwaite's method)
                df = (
                    (row1["SE"]**2 + row2["SE"]**2)**2 /
                    ((row1["SE"]**4 / (len(ls_means_df) - 1)) + (row2["SE"]**4 / (len(ls_means_df) - 1)))
                )
                
                # Compute p-value (two-tailed)
                p_value = 2 * t.sf(np.abs(t_stat), df)
                
                # Append the results to the list
                comparison_results.append({
                    "HeatStress_1": row1["HeatStress"],
                    "Parity_1": row1["Parity"],
                    "Breed_1": row1["Breed"],
                    "HeatStress_2": row2["HeatStress"],
                    "Parity_2": row2["Parity"],
                    "Breed_2": row2["Breed"],
                    "Mean_Difference": mean_diff,
                    "Pooled_SE": pooled_se,
                    "t_stat": t_stat,
                    "p_value": p_value
                })

    # Convert results into a DataFrame
    return pd.DataFrame(comparison_results)


# Perform pairwise comparisons for all combinations of HeatStress, Parity, and Breed
nested_comparisons = pairwise_nested_comparisons(cr_lsm)

# Save pairwise comparisons to CSV
nested_comparisons.to_csv("CR0_Pairwise_Comparisons.csv", index=False)

# Print the pairwise comparisons
print("\nPairwise Comparisons for Nested Effects (HeatStress, Parity, Breed):")
print(nested_comparisons)

Exempelkod analys med både HYS och SE_Number som slumpmässiga effekter

In [ ]:
# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CFI", "HeatStress", "LactationNumber", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["LactationNumber"] = df["LactationNumber"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# Define the formula for the mixed model
formula = "CFI ~ C(HeatStress) + C(LactationNumber) + C(Breed)"

# Fit the mixed model with HYS and CowID as random effects
model = smf.mixedlm(
    formula, 
    df, 
    groups=df["HYS"],  # Top-level random effect
    re_formula="~1",   # Random intercept
    vc_formula={"SE_Number": "0 + C(SE_Number)"}  # Nested random effect
)

# Fit the model
result = model.fit()
print(result.summary())

# CFI: LMM

In [ ]:
# --- PREP ---
df = pd.read_csv("../Data/fertility_filtered.csv", low_memory=False)
df = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CFI", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

df['logCFI'] = np.log(df['CFI'])

# --- FIT MODEL ---
# Define the formula for the mixed model
formula = "logCFI ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(formula, df, groups=df["HYS"])  # Random effect for HYS
result = model.fit()
print(result.summary())

# --- ESTIMATE LS-MEANS ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
print("Fixed effects:\n", fixed_effects)

# Extract the covariance matrix of fixed effects
cov_matrix = result.cov_re

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_CFI"] = predictions

# 1. LS-means for HeatStress
ls_means_heatstress = prediction_df.groupby("HeatStress")["Predicted_CFI"].mean()
print("\nLS-means estimates for HeatStress:\n", ls_means_heatstress)

# 2. LS-means for Parity
ls_means_parity = prediction_df.groupby("Parity")["Predicted_CFI"].mean()
print("\nLS-means estimates for Parity:\n", ls_means_parity)

# 3. LS-means for Breed
ls_means_breed = prediction_df.groupby("Breed")["Predicted_CFI"].mean()
print("\nLS-means estimates for Breed:\n", ls_means_breed)

# --- CALC SE ---

# Calculate standard errors for LS-means for each combination of HeatStress, Parity, and Breed
# This requires computing SE for each fixed effect level.

def calculate_se_for_factor(factor_name, factor_levels, fixed_effects, se_fixed_effects, reference_se=None):
    """
    Calculates standard errors for each level of a factor
    and returns them in a Series.
    """
    se_dict = {}
    for level in factor_levels:
        if level == factor_levels[0]:  # Assume the first level is the reference category
            # The SE for the reference category is the intercept SE
            se_dict[level] = reference_se
        else:
            # Construct the name of the fixed effect term for the factor
            effect_name = f"C({factor_name})[T.{level}]"
            # Extract the standard error for the fixed effect term
            se_value = se_fixed_effects.get(effect_name, np.nan)  # Default to NaN if not found
            se_dict[level] = se_value

    return pd.Series(se_dict, name=f"SE_{factor_name}")

# Calculate SE for each factor level
se_heatstress = calculate_se_for_factor("HeatStress", heatstress_levels, fixed_effects, se_fixed_effects, intercept_se)
se_parity = calculate_se_for_factor("Parity", parity_levels, fixed_effects, se_fixed_effects, intercept_se)
se_breed = calculate_se_for_factor("Breed", breed_levels, fixed_effects, se_fixed_effects, intercept_se)

# Print SE for each factor level
print("\nStandard Errors for HeatStress:\n", se_heatstress)
print("\nStandard Errors for Parity:\n", se_parity)
print("\nStandard Errors for Breed:\n", se_breed)

"""
# Save SEs to CSV
se_heatstress.to_csv("HeatStress_SE.csv")
se_parity.to_csv("Parity_SE.csv")
se_breed.to_csv("Breed_SE.csv")
"""

# --- PAIRWISE COMPARISONS (PDIFF EQUIVALENT IN SAS) ---

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_CFI"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_CFI"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

"""
# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)
"""

# --- EXAMINING RESIDUALS ---

# Calculate residuals
residuals = result.resid

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# CLI: LMM

In [ ]:
# --- PREP ---
df = pd.read_csv("../Data/fertility_filtered.csv", low_memory=False)
df = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CLI", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

df['logCLI'] = np.log(df['CLI'])

# --- FIT MODEL ---
# Define the formula for the mixed model
formula = "logCLI ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(formula, df, groups=df["HYS"])  # Random effect for HYS
result = model.fit()
print(result.summary())

# --- ESTIMATE LS-MEANS ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
print("Fixed effects:\n", fixed_effects)

# Extract the covariance matrix of fixed effects
cov_matrix = result.cov_re

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_CLI"] = predictions

# 1. LS-means for HeatStress
ls_means_heatstress = prediction_df.groupby("HeatStress")["Predicted_CLI"].mean()
print("\nLS-means estimates for HeatStress:\n", ls_means_heatstress)

# 2. LS-means for Parity
ls_means_parity = prediction_df.groupby("Parity")["Predicted_CLI"].mean()
print("\nLS-means estimates for Parity:\n", ls_means_parity)

# 3. LS-means for Breed
ls_means_breed = prediction_df.groupby("Breed")["Predicted_CLI"].mean()
print("\nLS-means estimates for Breed:\n", ls_means_breed)

# --- CALC SE ---

# Calculate standard errors for LS-means for each combination of HeatStress, Parity, and Breed
# This requires computing SE for each fixed effect level.

def calculate_se_for_factor(factor_name, factor_levels, fixed_effects, se_fixed_effects, reference_se=None):
    """
    Calculates standard errors for each level of a factor
    and returns them in a Series.
    """
    se_dict = {}
    for level in factor_levels:
        if level == factor_levels[0]:  # Assume the first level is the reference category
            # The SE for the reference category is the intercept SE
            se_dict[level] = reference_se
        else:
            # Construct the name of the fixed effect term for the factor
            effect_name = f"C({factor_name})[T.{level}]"
            # Extract the standard error for the fixed effect term
            se_value = se_fixed_effects.get(effect_name, np.nan)  # Default to NaN if not found
            se_dict[level] = se_value

    return pd.Series(se_dict, name=f"SE_{factor_name}")

# Calculate SE for each factor level
se_heatstress = calculate_se_for_factor("HeatStress", heatstress_levels, fixed_effects, se_fixed_effects, intercept_se)
se_parity = calculate_se_for_factor("Parity", parity_levels, fixed_effects, se_fixed_effects, intercept_se)
se_breed = calculate_se_for_factor("Breed", breed_levels, fixed_effects, se_fixed_effects, intercept_se)

# Print SE for each factor level
print("\nStandard Errors for HeatStress:\n", se_heatstress)
print("\nStandard Errors for Parity:\n", se_parity)
print("\nStandard Errors for Breed:\n", se_breed)

"""
# Save SEs to CSV
se_heatstress.to_csv("HeatStress_SE.csv")
se_parity.to_csv("Parity_SE.csv")
se_breed.to_csv("Breed_SE.csv")
"""

# --- PAIRWISE COMPARISONS (PDIFF EQUIVALENT IN SAS) ---

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_CLI"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_CLI"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

"""
# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)
"""

# --- EXAMINING RESIDUALS ---

# Calculate residuals
residuals = result.resid

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# FLI: LMM

In [ ]:
# --- PREP ---
df = pd.read_csv("../Data/fertility_filtered.csv", low_memory=False)
df = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["FLI", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# df['logFLI'] = np.log(df['FLI'].fillna(1) + 1)
df['logFLI'] = np.log(df['FLI'] + 1)

# --- FIT MODEL ---
# Define the formula for the mixed model
formula = "logFLI ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(formula, df, groups=df["HYS"])  # Random effect for HYS
result = model.fit()
print(result.summary())

# --- ESTIMATE LS-MEANS ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
print("Fixed effects:\n", fixed_effects)

# Extract the covariance matrix of fixed effects
cov_matrix = result.cov_re

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_FLI"] = predictions

# 1. LS-means for HeatStress
ls_means_heatstress = prediction_df.groupby("HeatStress")["Predicted_FLI"].mean()
print("\nLS-means estimates for HeatStress:\n", ls_means_heatstress)

# 2. LS-means for Parity
ls_means_parity = prediction_df.groupby("Parity")["Predicted_FLI"].mean()
print("\nLS-means estimates for Parity:\n", ls_means_parity)

# 3. LS-means for Breed
ls_means_breed = prediction_df.groupby("Breed")["Predicted_FLI"].mean()
print("\nLS-means estimates for Breed:\n", ls_means_breed)

# --- CALC SE ---

# Calculate standard errors for LS-means for each combination of HeatStress, Parity, and Breed
# This requires computing SE for each fixed effect level.

def calculate_se_for_factor(factor_name, factor_levels, fixed_effects, se_fixed_effects, reference_se=None):
    """
    Calculates standard errors for each level of a factor
    and returns them in a Series.
    """
    se_dict = {}
    for level in factor_levels:
        if level == factor_levels[0]:  # Assume the first level is the reference category
            # The SE for the reference category is the intercept SE
            se_dict[level] = reference_se
        else:
            # Construct the name of the fixed effect term for the factor
            effect_name = f"C({factor_name})[T.{level}]"
            # Extract the standard error for the fixed effect term
            se_value = se_fixed_effects.get(effect_name, np.nan)  # Default to NaN if not found
            se_dict[level] = se_value

    return pd.Series(se_dict, name=f"SE_{factor_name}")

# Calculate SE for each factor level
se_heatstress = calculate_se_for_factor("HeatStress", heatstress_levels, fixed_effects, se_fixed_effects, intercept_se)
se_parity = calculate_se_for_factor("Parity", parity_levels, fixed_effects, se_fixed_effects, intercept_se)
se_breed = calculate_se_for_factor("Breed", breed_levels, fixed_effects, se_fixed_effects, intercept_se)

# Print SE for each factor level
print("\nStandard Errors for HeatStress:\n", se_heatstress)
print("\nStandard Errors for Parity:\n", se_parity)
print("\nStandard Errors for Breed:\n", se_breed)

"""
# Save SEs to CSV
se_heatstress.to_csv("HeatStress_SE.csv")
se_parity.to_csv("Parity_SE.csv")
se_breed.to_csv("Breed_SE.csv")
"""

# --- PAIRWISE COMPARISONS (PDIFF EQUIVALENT IN SAS) ---

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_FLI"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_FLI"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

"""
# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)
"""

# --- EXAMINING RESIDUALS ---

# Calculate residuals
residuals = result.resid

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# NINS: LMM

In [ ]:
# --- PREP ---
df = pd.read_csv("../Data/fertility_filtered.csv", low_memory=False)
df = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["NINS", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# Natural-log transform
df['logNINS'] = np.log(df['NINS'] + 1)

# --- FIT MODEL ---
# Define the formula for the mixed model
formula = "logNINS ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(formula, df, groups=df["HYS"])  # Random effect for HYS
result = model.fit()
print(result.summary())

# --- ESTIMATE LS-MEANS ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
print("Fixed effects:\n", fixed_effects)

# Extract the covariance matrix of fixed effects
cov_matrix = result.cov_re

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_NINS"] = predictions

# 1. LS-means for HeatStress
ls_means_heatstress = prediction_df.groupby("HeatStress")["Predicted_NINS"].mean()
print("\nLS-means estimates for HeatStress:\n", ls_means_heatstress)

# 2. LS-means for Parity
ls_means_parity = prediction_df.groupby("Parity")["Predicted_NINS"].mean()
print("\nLS-means estimates for Parity:\n", ls_means_parity)

# 3. LS-means for Breed
ls_means_breed = prediction_df.groupby("Breed")["Predicted_NINS"].mean()
print("\nLS-means estimates for Breed:\n", ls_means_breed)

# --- CALC SE ---

# Calculate standard errors for LS-means for each combination of HeatStress, Parity, and Breed
# This requires computing SE for each fixed effect level.

def calculate_se_for_factor(factor_name, factor_levels, fixed_effects, se_fixed_effects, reference_se=None):
    """
    Calculates standard errors for each level of a factor
    and returns them in a Series.
    """
    se_dict = {}
    for level in factor_levels:
        if level == factor_levels[0]:  # Assume the first level is the reference category
            # The SE for the reference category is the intercept SE
            se_dict[level] = reference_se
        else:
            # Construct the name of the fixed effect term for the factor
            effect_name = f"C({factor_name})[T.{level}]"
            # Extract the standard error for the fixed effect term
            se_value = se_fixed_effects.get(effect_name, np.nan)  # Default to NaN if not found
            se_dict[level] = se_value

    return pd.Series(se_dict, name=f"SE_{factor_name}")

# Calculate SE for each factor level
se_heatstress = calculate_se_for_factor("HeatStress", heatstress_levels, fixed_effects, se_fixed_effects, intercept_se)
se_parity = calculate_se_for_factor("Parity", parity_levels, fixed_effects, se_fixed_effects, intercept_se)
se_breed = calculate_se_for_factor("Breed", breed_levels, fixed_effects, se_fixed_effects, intercept_se)

# Print SE for each factor level
print("\nStandard Errors for HeatStress:\n", se_heatstress)
print("\nStandard Errors for Parity:\n", se_parity)
print("\nStandard Errors for Breed:\n", se_breed)

"""
# Save SEs to CSV
se_heatstress.to_csv("HeatStress_SE.csv")
se_parity.to_csv("Parity_SE.csv")
se_breed.to_csv("Breed_SE.csv")
"""

# --- PAIRWISE COMPARISONS (PDIFF EQUIVALENT IN SAS) ---

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_NINS"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_NINS"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

"""
# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)
"""

# --- EXAMINING RESIDUALS ---

# Calculate residuals
residuals = result.resid

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# CI: LMM

In [ ]:
# --- PREP ---
df = pd.read_csv("../Data/fertility_filtered.csv", low_memory=False)
df = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])

# Ensure no missing values in the relevant columns
df = df.dropna(subset=["CI", "HeatStress", "Parity", "Breed", "HYS", "SE_Number"])

# Convert relevant columns to categorical variables
df = df.copy()
df["HeatStress"] = df["HeatStress"].astype("category")
df["Parity"] = df["Parity"].astype("category")
df["Breed"] = df["Breed"].astype("category")
df["HYS"] = df["HYS"].astype("category")
df["SE_Number"] = df["SE_Number"].astype("category")

# Natural-log transform
df['logCI'] = np.log(df['CI'])

# --- FIT MODEL ---
# Define the formula for the mixed model
formula = "logCI ~ C(HeatStress) + C(Parity) + C(Breed)"

# Fit the mixed model with HYS as a random effect
model = smf.mixedlm(formula, df, groups=df["HYS"])  # Random effect for HYS
# result = model.fit()
result = model.fit(method="lbfgs")
print(result.summary())

# --- ESTIMATE LS-MEANS ---

# Extract fixed effect coefficients (for LS-means)
fixed_effects = result.fe_params
print("Fixed effects:\n", fixed_effects)

# Extract the covariance matrix of fixed effects
cov_matrix = result.cov_re

# Create all combinations of HeatStress, Parity, and Breed
heatstress_levels = df["HeatStress"].cat.categories
parity_levels = df["Parity"].cat.categories
breed_levels = df["Breed"].cat.categories

combinations = list(product(heatstress_levels, parity_levels, breed_levels))
prediction_df = pd.DataFrame(combinations, columns=["HeatStress", "Parity", "Breed"])

# Match fixed effects structure for predictions
prediction_df["HeatStress"] = prediction_df["HeatStress"].astype("category")
prediction_df["Parity"] = prediction_df["Parity"].astype("category")
prediction_df["Breed"] = prediction_df["Breed"].astype("category")

# Predict CR0 values for each combination
predictions = result.predict(exog=prediction_df)

# Add predictions (LS-means) to the DataFrame
prediction_df["Predicted_CI"] = predictions

# 1. LS-means for HeatStress
ls_means_heatstress = prediction_df.groupby("HeatStress")["Predicted_CI"].mean()
print("\nLS-means estimates for HeatStress:\n", ls_means_heatstress)

# 2. LS-means for Parity
ls_means_parity = prediction_df.groupby("Parity")["Predicted_CI"].mean()
print("\nLS-means estimates for Parity:\n", ls_means_parity)

# 3. LS-means for Breed
ls_means_breed = prediction_df.groupby("Breed")["Predicted_CI"].mean()
print("\nLS-means estimates for Breed:\n", ls_means_breed)

# --- CALC SE ---

# Calculate standard errors for LS-means for each combination of HeatStress, Parity, and Breed
# This requires computing SE for each fixed effect level.

def calculate_se_for_factor(factor_name, factor_levels, fixed_effects, se_fixed_effects, reference_se=None):
    """
    Calculates standard errors for each level of a factor
    and returns them in a Series.
    """
    se_dict = {}
    for level in factor_levels:
        if level == factor_levels[0]:  # Assume the first level is the reference category
            # The SE for the reference category is the intercept SE
            se_dict[level] = reference_se
        else:
            # Construct the name of the fixed effect term for the factor
            effect_name = f"C({factor_name})[T.{level}]"
            # Extract the standard error for the fixed effect term
            se_value = se_fixed_effects.get(effect_name, np.nan)  # Default to NaN if not found
            se_dict[level] = se_value

    return pd.Series(se_dict, name=f"SE_{factor_name}")

# Calculate SE for each factor level
se_heatstress = calculate_se_for_factor("HeatStress", heatstress_levels, fixed_effects, se_fixed_effects, intercept_se)
se_parity = calculate_se_for_factor("Parity", parity_levels, fixed_effects, se_fixed_effects, intercept_se)
se_breed = calculate_se_for_factor("Breed", breed_levels, fixed_effects, se_fixed_effects, intercept_se)

# Print SE for each factor level
print("\nStandard Errors for HeatStress:\n", se_heatstress)
print("\nStandard Errors for Parity:\n", se_parity)
print("\nStandard Errors for Breed:\n", se_breed)

"""
# Save SEs to CSV
se_heatstress.to_csv("HeatStress_SE.csv")
se_parity.to_csv("Parity_SE.csv")
se_breed.to_csv("Breed_SE.csv")
"""

# --- PAIRWISE COMPARISONS (PDIFF EQUIVALENT IN SAS) ---

def pairwise_comparisons(ls_means_df, factor_name):
    """
    Perform pairwise comparisons for a given factor and return results.
    """
    levels = ls_means_df[factor_name].cat.categories
    comparison_results = []
    
    for i, level1 in enumerate(levels):
        for j, level2 in enumerate(levels):
            if i < j:  # Compare each pair only once
                group1 = ls_means_df[ls_means_df[factor_name] == level1]["Predicted_CI"]
                group2 = ls_means_df[ls_means_df[factor_name] == level2]["Predicted_CI"]
                t_stat, p_value = ttest_ind(group1, group2)
                comparison_results.append({
                    f"{factor_name}_1": level1,
                    f"{factor_name}_2": level2,
                    "t_stat": t_stat,
                    "p_value": p_value
                })
    
    return pd.DataFrame(comparison_results)

# Perform pairwise comparisons for HeatStress
heatstress_comparisons = pairwise_comparisons(prediction_df, "HeatStress")
print("\nPairwise Comparisons for HeatStress:")
print(heatstress_comparisons)

# Perform pairwise comparisons for Parity
parity_comparisons = pairwise_comparisons(prediction_df, "Parity")
print("\nPairwise Comparisons for Parity:")
print(parity_comparisons)

# Perform pairwise comparisons for Breed
breed_comparisons = pairwise_comparisons(prediction_df, "Breed")
print("\nPairwise Comparisons for Breed:")
print(breed_comparisons)

"""
# Save pairwise comparisons to CSV
heatstress_comparisons.to_csv("HeatStress_Pairwise_Comparisons.csv", index=False)
parity_comparisons.to_csv("Parity_Pairwise_Comparisons.csv", index=False)
breed_comparisons.to_csv("Breed_Pairwise_Comparisons.csv", index=False)
"""

# --- EXAMINING RESIDUALS ---

# Calculate residuals
residuals = result.resid

# Plot residuals
plt.figure(figsize=(10, 6))

# 1. Residuals vs Fitted values plot
fitted_values = result.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

# 2. Q-Q plot for normality check
import scipy.stats as stats
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot for Residuals")
plt.show()

# 3. Histogram of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print(result.cov_re)